In [1]:
import os
import sys
import boto3
import json
import numpy as np
import scipy.io.wavfile as sciwav
import IPython.display
import shutil

from magenta.models.nsynth import utils
from magenta.models.nsynth.wavenet.fastgen import encode
import numpy as np
import tensorflow.compat.v1 as tf

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [2]:
# set parameters
bucket = 'music-ml-gigioli'
checkpoint_path = 'wavenet-ckpt/model.ckpt-200000'
batch_size = 16
sample_length = 64000

In [3]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key='data/nsynth/nsynth-train/examples.json')

train_examples = json.loads(obj['Body'].read().decode('utf-8'))

wavfiles = list(train_examples.keys())

In [4]:
ignore = [f.key for f in boto3.resource('s3').Bucket(bucket).objects.filter(Prefix='data/nsynth/nsynth-train/embeddings/').all()]
ignore = [x.split('/')[-1].split('_embeddings.npy')[0] for x in ignore]

ct0 = len(wavfiles)
wavfiles = [x for x in wavfiles if x not in set(ignore)]
ct1 = len(wavfiles)
print('Ignoring {} wav files'.format(ct0-ct1))

Ignoring 3360 wav files


In [ ]:
# create local directories to store temporary copies
if os.path.exists('/tmp/nsynth_wav_files'):
    shutil.rmtree('/tmp/nsynth_wav_files')
os.mkdir('/tmp/nsynth_wav_files')

if os.path.exists('/tmp/nsynth_embeddings'):
    shutil.rmtree('/tmp/nsynth_embeddings')
os.mkdir('/tmp/nsynth_embeddings')

# loop through wav files in batches
for start_file in range(0, len(wavfiles), batch_size):
    batch_number = (start_file / batch_size) + 1
    tf.logging.info("On file number %s (batch %d).", start_file, batch_number)
    end_file = start_file + batch_size
    batch_keys = wavfiles[start_file:end_file]
    
    batch_keys = ['data/nsynth/nsynth-train/audio/{}.wav'.format(x) for x in batch_keys]
    
    # download wav files locally
    for key in batch_keys:
        s3.download_file(bucket, key, '/tmp/nsynth_wav_files/{}'.format(key.split('/')[-1]))
        
    wavefiles_batch = ['/tmp/nsynth_wav_files/{}'.format(key.split('/')[-1]) for key in batch_keys]

    # Ensure that files has batch_size elements.
    batch_filler = batch_size - len(wavefiles_batch)
    wavefiles_batch.extend(batch_filler * [wavefiles_batch[-1]])
    wav_data = [utils.load_audio(f, sample_length) for f in wavefiles_batch]
    min_len = min([x.shape[0] for x in wav_data])
    wav_data = np.array([x[:min_len] for x in wav_data])
    
    try:
        tf.reset_default_graph()
        
        # Load up the model for encoding and find the encoding
        encoding = encode(wav_data, checkpoint_path, sample_length=sample_length)
        if encoding.ndim == 2:
            encoding = np.expand_dims(encoding, 0)

        tf.logging.info("Encoding:")
        tf.logging.info(encoding.shape)
        tf.logging.info("Sample length: %d" % sample_length)

        for num, (wavfile, enc) in enumerate(zip(wavefiles_batch, encoding)):
            filename = "%s_embeddings.npy" % wavfile.split("/")[-1].replace('.wav', '')
            path = os.path.join('/tmp/nsynth_embeddings', filename)
            
            # save locally
            np.save(path, enc)
            
            # upload to s3
            s3.upload_file(path, bucket, 'data/nsynth/nsynth-train/embeddings/{}'.format(filename))

            if num + batch_filler + 1 == batch_size:
                break
    except Exception as e:
        tf.logging.info("Unexpected error happened: %s.", e)
        raise

    # delete local embedding copies
    for key in batch_keys:
        os.remove('/tmp/nsynth_embeddings/{}_embeddings.npy'.format(key.split('/')[-1].replace('.wav', '')))
    
    # delete local wav file copies
    for key in batch_keys:
        os.remove('/tmp/nsynth_wav_files/{}'.format(key.split('/')[-1]))

INFO:tensorflow:On file number 0 (batch 1).
INFO:tensorflow:Restoring parameters from wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Encoding:
INFO:tensorflow:(16, 125, 16)
INFO:tensorflow:Sample length: 64000
INFO:tensorflow:On file number 16 (batch 2).
INFO:tensorflow:Restoring parameters from wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Encoding:
INFO:tensorflow:(16, 125, 16)
INFO:tensorflow:Sample length: 64000
INFO:tensorflow:On file number 32 (batch 3).
INFO:tensorflow:Restoring parameters from wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Encoding:
INFO:tensorflow:(16, 125, 16)
INFO:tensorflow:Sample length: 64000
INFO:tensorflow:On file number 48 (batch 4).
INFO:tensorflow:Restoring parameters from wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Encoding:
INFO:tensorflow:(16, 125, 16)
INFO:tensorflow:Sample length: 64000
INFO:tensorflow:On file number 64 (batch 5).
INFO:tensorflow:Restoring parameters from wavenet-ckpt/model.ckpt-200000
INFO:tensorflow:Encoding:
INFO:tensorfl

### Load embeddings from s3

In [39]:
from s3fs.core import S3FileSystem
s3 = S3FileSystem()

In [52]:
data = np.load(
    s3.open('{}/{}'.format(bucket, 'data/nsynth/nsynth-train/embeddings/bass_synthetic_079-048-050_embeddings.npy'))
)

In [53]:
data.shape

(125, 16)